In [1]:
import os
import csv
import pandas as pd
import numpy as np
from matplotlib import pyplot
import math

In [2]:

df = pd.DataFrame()
header_first = True
mode='w'
for root, dirs, files in os.walk('results'):
    for f in files: 
        if(f.startswith('Result_')):
          file_name=os.path.join(root, f)
          model = f.split('_')[1]
          frequence = f.split('_')[3].split('.')[0]
          dataset = pd.read_csv(file_name, sep =';', encoding = 'latin1', decimal='.')
          dataset['model']=model
          dataset['freq']=frequence
          dataset['freq'].replace(['Hour','Day','Week'],['1-Hour','2-Day','3-Week'],inplace=True)
          dataset.columns=[ 'SK_PONTO','Best_Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration','model','freq']
          dataset.to_csv('tables/all_results.csv',sep=';',mode=mode,index=False, header=header_first)
          header_first = False
          mode='a'
        


In [4]:
dataset = pd.read_csv(f'tables/all_results.csv', sep =';', encoding = 'latin1', decimal='.')
dataset['MAE']=round(dataset['MAE'].str.replace(',','.').astype(float),2)
dataset['MSE']=round(dataset['MSE'].str.replace(',','.').astype(float),2)
dataset['RMSE']=round(dataset['RMSE'].str.replace(',','.').astype(float),2)
dataset['MAPE']=round(dataset['MAPE'].str.replace(',','.').astype(float),2)
dataset['Duration']=round(dataset['Duration'].str.replace(',','.').astype(float),1)

fs=dataset.pop('freq')
dataset.insert(1, 'freq', fs)

fs=dataset.pop('model')
dataset.insert(3, 'model', fs)

fs=dataset.pop('Best_Params')
dataset.insert(9, 'Best_Params', fs)

idx = dataset.groupby(['SK_PONTO','freq'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].sort_values(['SK_PONTO','freq']).to_csv('tables/best_model_per_reservior.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="2-Day"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_model_per_reservior_day.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="1-Hour"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_model_per_reservior_hour.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="3-Week"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_model_per_reservior_week.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="2-Day"').sort_values(['SK_PONTO','freq']).head()

,SK_PONTO,freq,model,N_Past_Vl,MSE,RMSE,MAE,MAPE,Duration,Best_Params
183,1,2-Day,ARIMA2,4,3972.49,63.03,43.61,6.64,146.6,"ARIMA(3,1,0)(0,0,0)"
326,4,2-Day,LSTM,2,3960186.53,1990.02,330.86,8.64,8.8,"LSTM(100,96,90)"
333,5,2-Day,LSTM,3,3524767.56,1877.44,669.97,8.35,9.1,"LSTM(100,96,90)"
520,6,2-Day,MLP,4,260528.70,510.42,195.76,5.46,38.6,"{'activation': 'identity', 'alpha': 0,0001, 'h..."
204,7,2-Day,ARIMA2,1,2207.54,46.98,36.44,6.45,31.1,"ARIMA(0,1,1)(0,0,0)"


In [5]:
idx = dataset.groupby(['SK_PONTO','freq','model'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].query('model=="SVR-LSTM" and freq=="1-Hour"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrsltm_best_model_per_reservior_hour.csv',sep=';',decimal=',', index=False)
dataset[idx].query('model=="SVR-LSTM" and freq=="2-Day"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrsltm_best_model_per_reservior_day.csv',sep=';',decimal=',', index=False)
dataset[idx].query('model=="SVR-LSTM" and freq=="3-Week"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrsltm_best_model_per_reservior_week.csv',sep=';',decimal=',', index=False)


In [6]:
idx = dataset.groupby(['SK_PONTO','freq','model'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].query('model=="SVR-MLP" and freq=="1-Hour"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrmlp_best_model_per_reservior_hour.csv',sep=';',decimal=',', index=False)
dataset[idx].query('model=="SVR-MLP" and freq=="2-Day"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrmlp_best_model_per_reservior_day.csv',sep=';',decimal=',', index=False)
dataset[idx].query('model=="SVR-MLP" and freq=="3-Week"').sort_values(['SK_PONTO','freq']).to_csv('tables/svrmlp_best_model_per_reservior_week.csv',sep=';',decimal=',', index=False)

In [7]:
dataset['SVR-LSTM']=np.where(dataset['model']=='SVR-LSTM',dataset['MAPE'],999)
dataset['SVR-MLP']=np.where(dataset['model']=='SVR-MLP',dataset['MAPE'],999)
dataset['SVR']=np.where(dataset['model']=='SVR',dataset['MAPE'],999)
dataset['LSTM']=np.where(dataset['model']=='LSTM',dataset['MAPE'],999)
dataset['ARIMA1']=np.where(dataset['model']=='ARIMA1',dataset['MAPE'],999)
dataset['ARIMA2']=np.where(dataset['model']=='ARIMA2',dataset['MAPE'],999)
dataset['MLP']=np.where(dataset['model']=='MLP',dataset['MAPE'],999)

In [8]:
idx = dataset.groupby(['SK_PONTO','freq','SVR-LSTM','SVR-MLP','SVR','MLP','LSTM','ARIMA1','ARIMA2'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].query('freq=="1-Hour"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_mape_per_reservior_hour.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="2-Day"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_mape_per_reservior_day.csv',sep=';',decimal=',', index=False)
dataset[idx].query('freq=="3-Week"').sort_values(['SK_PONTO','freq']).to_csv('tables/best_mape_per_reservior_week.csv',sep=';',decimal=',', index=False)

In [9]:
dataset2=dataset.query('freq=="1-Hour"').groupby(['SK_PONTO','freq']).min(['SVR','LSTM','ARIMA1','ARIMA2','MLP','SVR-LSTM','SVR-MLP'])
dataset2

,,N_Past_Vl,MSE,RMSE,MAE,MAPE,Duration,SVR-LSTM,SVR-MLP,SVR,LSTM,ARIMA1,ARIMA2,MLP
SK_PONTO,freq,,,,,,,,,,,,,
1,1-Hour,1,1613.21,40.16,5.19,2.31,1.2,3.54,3.32,28.94,2.31,6.69,6.64,3.25
4,1-Hour,1,2186.08,46.76,18.09,4.93,2.0,10.13,11.96,15.70,4.93,11.88,11.85,9.02
5,1-Hour,1,3923.18,62.64,29.23,4.85,2.4,7.17,5.82,9.75,4.85,17.89,20.87,6.03
6,1-Hour,1,4175.91,64.62,28.15,8.00,2.3,34.41,34.32,8.00,84.53,38.02,52.47,34.69
7,1-Hour,1,1193.79,34.55,15.56,3.95,2.5,6.09,4.70,16.46,3.95,6.51,6.45,5.04
8,1-Hour,1,21.10,4.59,1.43,1.20,1.9,2.27,1.33,3.28,1.20,7.02,3.34,1.35
9,1-Hour,1,50.90,7.13,2.81,0.94,1.2,2.04,0.95,4.39,0.94,1.05,8.52,0.95
11,1-Hour,1,426.60,20.65,7.70,1.51,2.8,1.82,1.54,6.86,1.53,1.63,3.24,1.51
12,1-Hour,1,1216.21,34.87,15.68,6.33,2.8,9.87,8.09,6.33,7.34,12.61,11.71,8.01


In [14]:
dataset['SVR-LSTM']=np.where(dataset['model']=='SVR-LSTM',dataset['Duration'],999)
dataset['SVR-MLP']=np.where(dataset['model']=='SVR-MLP',dataset['Duration'],999)
dataset['SVR']=np.where(dataset['model']=='SVR',dataset['Duration'],999)
dataset['LSTM']=np.where(dataset['model']=='LSTM',dataset['Duration'],999)
dataset['ARIMA1']=np.where(dataset['model']=='ARIMA1',dataset['Duration'],999)
dataset['ARIMA2']=np.where(dataset['model']=='ARIMA2',dataset['Duration'],999)
dataset['MLP']=np.where(dataset['model']=='MLP',dataset['Duration'],999)
dataset2=dataset.query('freq=="1-Hour"').groupby(['SK_PONTO','freq']).min(['SVR','LSTM','ARIMA1','ARIMA2','MLP','SVR-LSTM','SVR-MLP'])
dataset2

,,N_Past_Vl,MSE,RMSE,MAE,MAPE,Duration,SVR-LSTM,SVR-MLP,SVR,LSTM,ARIMA1,ARIMA2,MLP
SK_PONTO,freq,,,,,,,,,,,,,
1,1-Hour,1,1613.21,40.16,5.19,2.31,1.2,98.3,167.6,1.2,79.3,45.5,41.1,123.5
4,1-Hour,1,2186.08,46.76,18.09,4.93,2.0,162.4,202.4,2.0,144.4,18.6,26.4,209.0
5,1-Hour,1,3923.18,62.64,29.23,4.85,2.4,148.2,204.3,2.4,118.3,37.5,18.5,198.7
6,1-Hour,1,4175.91,64.62,28.15,8.00,2.3,166.8,254.0,2.3,243.1,28.9,14.1,210.6
7,1-Hour,1,1193.79,34.55,15.56,3.95,2.5,144.3,175.5,2.5,288.9,36.0,31.6,217.0
8,1-Hour,1,21.10,4.59,1.43,1.20,1.9,155.3,182.6,1.9,118.7,13.9,23.4,191.5
9,1-Hour,1,50.90,7.13,2.81,0.94,1.2,74.1,102.6,1.2,58.1,10.4,7.5,128.7
11,1-Hour,1,426.60,20.65,7.70,1.51,2.8,156.9,180.6,2.8,151.9,40.9,90.3,219.1
12,1-Hour,1,1216.21,34.87,15.68,6.33,2.8,161.4,205.4,2.8,156.2,67.8,63.5,186.4


In [ ]:
idx=dataset.groupby(['SK_PONTO','model','freq','MAPE'])['MAPE'].transform(min)==dataset['MAPE']
dataset[idx].sort_values(['SK_PONTO','freq']).to_csv('results/best_model_per_reservior.csv',sep=';',decimal=',', index=False)